# TensorFunction

In [1]:
import collections
import numpy as np
import pandas as pd
import tensorflow as tf
if not hasattr(tf, "placeholder"):
    import tensorflow.compat.v1 as tf
    tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [39]:
class TensorFunction(object):

    def __init__(self, *args, **kwargs) -> None:
        """ Initialize TensorFunction class. """
        self.Fitted = False
        self.Function = None
        self.X = tf.placeholder(dtype=tf.float32)
        self.Y = tf.placeholder(dtype=tf.float32)
        self.Params = {}
    
    def _set_parameters(self, params: str) -> None:
        """ Dynamically set function parameters. """
        variable_code = "self.{0} = tf.Variable({1}, name='{0}', dtype=tf.float32)"
        for k, v in params.items():
            byte_code = compile(variable_code.format(k, v), filename="<inline code>", mode="exec")
            exec(byte_code)
        return
    
    def _set_function(self, func: str) -> None:
        """ Dynamically set TensorFunction. """
        function_code = "self.Function = {0}"
        byte_code = compile(function_code.format(func), filename="<inline code>", mode="exec")
        exec(byte_code)
        return
    
    def initialize(self, func: str, params: dict) -> object:
        """ Initialize Function Parameters. """
        if self.Function is not None:
            raise TypeError("TensorFunction has already been initialized. Please reset before initializing again.")
        if not params:
            raise ValueError("params cannot be an empty dict.")
        if not self.Fitted:
            self._set_parameters(params)
        if not self.Function:
            self._set_function(func)
            if self.Function is None:
                raise ValueError("self.Function is NoneType.")
        return self
    
    def reset(self) -> object:
        """ Reset attributes. """
        for k, v in self.__dict__.copy().items():
            if isinstance(v, tf.Variable):
                del self.__dict__[k]
                
        self.Fitted = False
        self.Function = None
        self.X = tf.placeholder(dtype=tf.float32)
        self.Y = tf.placeholder(dtype=tf.float32)
        self.Params = {}
        return self 

    def fit(self, x: np.ndarray, y: np.ndarray, metric: str = "mse", learning_rate: float = 0.01, 
            num_rounds: int = 100, early_stopping_rounds: int = 0, verbose_eval: int = 0, 
            random_state: int = 0) -> object:
        """ Fit TensorFunction to array-like data. """
        np.random.seed(int(random_state))
        try:
            x = np.array(x, dtype=np.float32)
            y = np.array(y, dtype=np.float32)
            assert x.ndim == y.ndim == 1
            assert x.shape[0] == y.shape[0]
        except Exception as e:
            raise ValueError("x and y must be array-like objects: {0}".format(e))
            
        if self.Function is None:
            raise ValueError("TensorFunction has not been initialized. Please initialize before fitting.")
            
        if metric == "mse":
            loss = tf.reduce_mean(tf.square(self.Y - self.Function))
        else:
            raise ValueError("'{0}' is not a valid error metric.".format(metric))
            
        train = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
        
        if 1 < int(early_stopping_rounds) < 1000:
            print("Fitting until error does not improve for {0} rounds.".format(int(early_stopping_rounds)))
            error_rounds = collections.deque(maxlen=int(early_stopping_rounds))
        else:
            error_rounds = None
            
        with tf.Session() as sess:
            tf.global_variables_initializer().run()
            for episode in range(int(num_rounds)):
                for (i, j) in zip(x, y):
                    sess.run(train, feed_dict={self.X: i, self.Y: j})
                error = sess.run(loss, feed_dict={self.X: i, self.Y: j})
                if verbose_eval > 0 and episode % verbose_eval == 0:
                    print("[Episode - {0}] {1}: {2:,.8f}".format(episode + 1, metric, error))
                if error_rounds is not None:
                    error_rounds.append(error)
                    if len(error_rounds) == int(early_stopping_rounds):
                        if all(e == error_rounds[0] for e in error_rounds):
                            print("Early stopping, best iteration is:")
                            print("[Episode - {0}] {1}: {2:,.8f}".format((episode + 1) - int(early_stopping_rounds), metric, error))
                            break
            for k, v in self.__dict__.copy().items():
                if isinstance(v, tf.Variable):
                    self.Params[k] = sess.run(v)
        self.fitted = True
        return self

In [40]:
tfunc = TensorFunction()

In [41]:
tfunc.initialize(
    func="self.Function = tf.add(tf.multiply(self.m, self.X), self.b)", 
    params=dict(m=0., b=0.)
);

In [42]:
X = np.linspace(-1, 1, 100)
y = 5. * X + 0.33 * np.random.randn(*X.shape)
tfunc.fit(X, y, verbose_eval=1, early_stopping_rounds=10)

Fitting until error does not improve for 10 rounds.
[Episode - 1] mse: 4.46380854
[Episode - 2] mse: 1.08609748
[Episode - 3] mse: 0.31144872
[Episode - 4] mse: 0.10223599
[Episode - 5] mse: 0.03982750
[Episode - 6] mse: 0.01920754
[Episode - 7] mse: 0.01157801
[Episode - 8] mse: 0.00843569
[Episode - 9] mse: 0.00702849
[Episode - 10] mse: 0.00636197
[Episode - 11] mse: 0.00603576
[Episode - 12] mse: 0.00587327
[Episode - 13] mse: 0.00579163
[Episode - 14] mse: 0.00575011
[Episode - 15] mse: 0.00572938
[Episode - 16] mse: 0.00571870
[Episode - 17] mse: 0.00571329
[Episode - 18] mse: 0.00571048
[Episode - 19] mse: 0.00570911
[Episode - 20] mse: 0.00570832
[Episode - 21] mse: 0.00570774
[Episode - 22] mse: 0.00570753
[Episode - 23] mse: 0.00570745
[Episode - 24] mse: 0.00570738
[Episode - 25] mse: 0.00570738
[Episode - 26] mse: 0.00570731
[Episode - 27] mse: 0.00570724
[Episode - 28] mse: 0.00570724
[Episode - 29] mse: 0.00570717
[Episode - 30] mse: 0.00570717
[Episode - 31] mse: 0.00570

In [43]:
tfunc.Params

{'m': 5.0388403, 'b': 0.018270273}

In [8]:
x = np.linspace(0, 10, 100)

In [13]:
y = 8 * np.sin(x) * np.cos(x) + 0.45 * np.random.randn(*x.shape)

In [11]:
x

array([ 0.        ,  0.1010101 ,  0.2020202 ,  0.3030303 ,  0.4040404 ,
        0.50505051,  0.60606061,  0.70707071,  0.80808081,  0.90909091,
        1.01010101,  1.11111111,  1.21212121,  1.31313131,  1.41414141,
        1.51515152,  1.61616162,  1.71717172,  1.81818182,  1.91919192,
        2.02020202,  2.12121212,  2.22222222,  2.32323232,  2.42424242,
        2.52525253,  2.62626263,  2.72727273,  2.82828283,  2.92929293,
        3.03030303,  3.13131313,  3.23232323,  3.33333333,  3.43434343,
        3.53535354,  3.63636364,  3.73737374,  3.83838384,  3.93939394,
        4.04040404,  4.14141414,  4.24242424,  4.34343434,  4.44444444,
        4.54545455,  4.64646465,  4.74747475,  4.84848485,  4.94949495,
        5.05050505,  5.15151515,  5.25252525,  5.35353535,  5.45454545,
        5.55555556,  5.65656566,  5.75757576,  5.85858586,  5.95959596,
        6.06060606,  6.16161616,  6.26262626,  6.36363636,  6.46464646,
        6.56565657,  6.66666667,  6.76767677,  6.86868687,  6.96

In [12]:
y

array([ 0.        ,  0.80259543,  1.57254645,  2.27853643,  2.89185025,
        3.38754225,  3.7454509 ,  3.95101877,  3.99588469,  3.8782238 ,
        3.60282178,  3.18088023,  2.62956099,  1.97128817,  1.23283607,
        0.44424015, -0.36242459, -1.15434823, -1.89932044, -2.56704055,
       -3.13035001, -3.56633703, -3.85726847, -3.99131111, -3.96301297,
       -3.77352503, -3.43055444, -2.94805103, -2.34563993, -1.64782332,
       -0.8829839 , -0.08223039,  0.72186773,  1.49660492,  2.21046988,
        2.83442719,  3.34309829,  3.71579372,  3.93735463,  3.99876936,
        3.89753996,  3.63778377,  3.23006602,  2.69097001,  2.04242271,
        1.31080284,  0.52586795, -0.28045584, -1.0753725 , -1.82654995,
       -2.50343513, -3.07849672, -3.52834493, -3.83468283, -3.98505057,
       -3.97333217, -3.80000425, -3.47211668, -3.0030058 , -2.41175202,
       -1.72240373, -0.96299917, -0.16442601,  0.64083493,  1.42003084,
        2.14146906,  2.77580614,  3.29724133,  3.68456602,  3.92